In [2]:
import serial
import csv
import os
from datetime import datetime, timedelta
import time

In [3]:

def reset_esp32(port, baudrate=921600):
    """Resets an ESP32 using the RTS and DTR lines of the serial port.

    Args:
        port: The serial port (e.g., "COM3" on Windows, "/dev/ttyUSB0" on Linux).
        baudrate: The baud rate (typically 115200 for communication,
                  but can be higher for flashing).
    """
    try:
        with serial.Serial(port, baudrate) as ser:
            # Method 1 (esptool.py compatible):
            ser.dtr = False  # DTR = 0: Set EN (reset) to HIGH
            ser.rts = True   # RTS = 1, Set IO0 (GPIO0) to LOW: BOOT mode
            time.sleep(0.1)
            ser.dtr = True   # DTR = 1, Set EN to LOW: Reset chip
            time.sleep(0.1)  # Wait for the chip to reset
            ser.dtr = False  # DTR = 0, set EN to HIGH
            ser.rts = False # Set IO0 to HIGH, back to normal mode.
            time.sleep(0.1)

            # Method 2 (Simple DTR toggle - might work in some cases):
            # ser.dtr = True   # Assert DTR (often connected to reset)
            # time.sleep(0.1)
            # ser.dtr = False  # De-assert DTR
            # time.sleep(0.1)
            print(f"ESP32 reset via DTR/RTS on port {port}")

    except serial.SerialException as e:
        print(f"Error resetting ESP32: {e}")


In [4]:
def collect_csi_data(filepath, port, baudrate = 921600, seconds = 100, minutes = 0, hours = 0, temp_data = False):
    with serial.Serial(port, baudrate) as ser:
        header = "type,seq,mac,rssi,rate,sig_mode,mcs,bandwidth,smoothing,not_sounding,aggregation,stbc,fec_coding,sgi,noise_floor,ampdu_cnt,channel,secondary_channel,local_timestamp,ant,sig_len,rx_state,len,first_word,data,datetime"
        
        start_time = datetime.now()
        
        filename = filepath
        if temp_data:
            temp_file_path = temp_data
        
        if not os.path.exists(filename):
            with open(filename, 'w', encoding = 'utf-8') as f:
                f.write(header + '\n')
                
        
        if ser.in_waiting > 0:
            ser.reset_input_buffer()
        
        num_collected = 0
        
        while (datetime.now() - start_time) <= timedelta(seconds = seconds, minutes = minutes, hours = hours):
            if ser.in_waiting > 0:
                try:
                    line = ser.readline().decode('utf-8').strip()
                except:
                    print("DECODE FAILED")
                    continue
                line = line + f',{datetime.now()}'
                if not line.startswith("CSI_DATA"):
                    continue
                with open(filename, 'a', newline = '', encoding = 'utf-8') as f:
                    f.write(line + '\n')       
        
                num_collected += 1

                if temp_data:
                    if num_collected % 5 == 0:
                        with open(temp_file_path, 'w', newline = '', encoding = 'utf-8') as f:
                            f.write(header + '\n')
                            f.write(line + '\n')  
                
                if num_collected % 100 == 0:
                    print(f"{datetime.now()} Number of CSI data collected : {num_collected}")
                    print(f"ser_inwaiting : {ser.in_waiting}")
        if temp_data:
            os.remove(temp_file_path)
        
        print(f"Total : {num_collected}")
    

In [5]:
PORT = "COM9"
BAUDRATE = "921600"
FILEPATH = "ly8_csi_har_dataset/valid/1.csv"


reset_esp32(PORT, BAUDRATE)
time.sleep(10)
collect_csi_data(FILEPATH, PORT, BAUDRATE, seconds = 0, minutes = 3)

ESP32 reset via DTR/RTS on port COM9
2025-03-31 10:53:50.863637 Number of CSI data collected : 100
ser_inwaiting : 0
2025-03-31 10:53:54.165459 Number of CSI data collected : 200
ser_inwaiting : 0
2025-03-31 10:53:57.393932 Number of CSI data collected : 300
ser_inwaiting : 0
2025-03-31 10:54:00.592392 Number of CSI data collected : 400
ser_inwaiting : 0
2025-03-31 10:54:03.762478 Number of CSI data collected : 500
ser_inwaiting : 0
2025-03-31 10:54:06.998609 Number of CSI data collected : 600
ser_inwaiting : 0
2025-03-31 10:54:10.225741 Number of CSI data collected : 700
ser_inwaiting : 0
2025-03-31 10:54:13.467493 Number of CSI data collected : 800
ser_inwaiting : 0
2025-03-31 10:54:16.761691 Number of CSI data collected : 900
ser_inwaiting : 0
2025-03-31 10:54:19.931250 Number of CSI data collected : 1000
ser_inwaiting : 0
2025-03-31 10:54:23.097625 Number of CSI data collected : 1100
ser_inwaiting : 0
2025-03-31 10:54:26.302240 Number of CSI data collected : 1200
ser_inwaiting : 0
